In [1]:
import logging
import os

api_key = os.environ["PALM_API_KEY"]
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.google_palm as sk_gp

kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "models/chat-bison-001",
    sk_gp.GooglePalmChatCompletion("models/chat-bison-001", api_key),
)

/Users/alextanhongpin/Documents/python/python-palm-semantic-kernel/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
ask = "convert this text to uppercase: she sells seashells by the seashore"

## Without skills

In [17]:
from semantic_kernel.connectors.ai.complete_request_settings import (
    CompleteRequestSettings,
)

chat_completion = sk_gp.GooglePalmChatCompletion("models/chat-bison-001", api_key)
settings = CompleteRequestSettings()
completion = await chat_completion.complete_async(ask, settings)
print(completion)

SHE SELLS SEASHELLS BY THE SEASHORE.


In [35]:
completion = await chat_completion.complete_async(
    "what is the ceil(1.23 + 3.21)", settings
)
print(completion)

The ceil function rounds a number up to the nearest integer. In this case, the ceil of 1.23 + 3.21 is 4.


## Basic Planner

In [5]:
from semantic_kernel.core_skills.text_skill import TextSkill

text_skill = kernel.import_skill(TextSkill(), "TextSkill")

DEBUG:root:Importing skill TextSkill
DEBUG:root:Methods imported: 5


In [7]:
from semantic_kernel.planning.basic_planner import BasicPlanner

planner = BasicPlanner()

In [18]:
basic_plan = await planner.create_plan_async(ask, kernel)
basic_plan

DEBUG:root:Extracting blocks from template: 
You are a planner for the Semantic Kernel.
Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
Create a list of subtasks based off the [GOAL] provided.
Each subtask must be from within the [AVAILABLE FUNCTIONS] list. Do not use any functions that are not in the list.
Base your decisions on which functions to use from the description and the name of the function.
Sometimes, a function may take arguments. Provide them if necessary.
The plan should be as short as possible.
For example:

[AVAILABLE FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterSkill.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterSkill.Translate
description: translate the input to an

Prompt: 
You are a planner for the Semantic Kernel.
Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
Create a list of subtasks based off the [GOAL] provided.
Each subtask must be from within the [AVAILABLE FUNCTIONS] list. Do not use any functions that are not in the list.
Base your decisions on which functions to use from the description and the name of the function.
Sometimes, a function may take arguments. Provide them if necessary.
The plan should be as short as possible.
For example:

[AVAILABLE FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterSkill.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterSkill.Translate
description: translate the input to another language
args:
- input: the te

In [19]:
print(basic_plan.generated_plan)

The following is a JSON plan step by step, to satisfy the goal given.

[GOAL]
convert this text to uppercase: she sells seashells by the seashore

[OUTPUT]

```json
{
  "input": "she sells seashells by the seashore",
  "subtasks": [
    {
      "function": "TextSkill.uppercase",
      "args": {
        "input": "she sells seashells by the seashore"
      }
    }
  ]
}
```


In [20]:
results = await planner.execute_plan_async(basic_plan, kernel)
print(results)

SHE SELLS SEASHELLS BY THE SEASHORE


## Sequential Planner

In [22]:
from semantic_kernel.planning import SequentialPlanner

In [23]:
planner = SequentialPlanner(kernel)

DEBUG:root:Extracting blocks from template: Create an XML plan step by step, to satisfy the goal given, with the available functions.

[AVAILABLE FUNCTIONS]

{{$available_functions}}

[END AVAILABLE FUNCTIONS]

To create a plan, follow these steps:
0. The plan should be as short as possible.
1. From a <goal> create a <plan> as a series of <functions>.
2. A plan has 'INPUT' available in context variables by default.
3. Before using any function in a plan, check that it is present in the [AVAILABLE FUNCTIONS] list. If it is not, do not use it.
4. Only use functions that are required for the given goal.
5. Append an "END" XML comment at the end of the plan after the final closing </plan> tag.
6. Always output valid XML that can be parsed by an XML parser.
7. If a plan cannot be created with the [AVAILABLE FUNCTIONS], return <plan />.

All plans take the form of:
<plan>
    <!-- ... reason for taking step ... -->
    <function.{FullyQualifiedFunctionName} ... />
    <!-- ... reason for tak

In [24]:
sequential_plan = await planner.create_plan_async(goal=ask)

DEBUG:root:Rendering string template: Create an XML plan step by step, to satisfy the goal given, with the available functions.

[AVAILABLE FUNCTIONS]

{{$available_functions}}

[END AVAILABLE FUNCTIONS]

To create a plan, follow these steps:
0. The plan should be as short as possible.
1. From a <goal> create a <plan> as a series of <functions>.
2. A plan has 'INPUT' available in context variables by default.
3. Before using any function in a plan, check that it is present in the [AVAILABLE FUNCTIONS] list. If it is not, do not use it.
4. Only use functions that are required for the given goal.
5. Append an "END" XML comment at the end of the plan after the final closing </plan> tag.
6. Always output valid XML that can be parsed by an XML parser.
7. If a plan cannot be created with the [AVAILABLE FUNCTIONS], return <plan />.

All plans take the form of:
<plan>
    <!-- ... reason for taking step ... -->
    <function.{FullyQualifiedFunctionName} ... />
    <!-- ... reason for taking st

In [26]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Convert a string to uppercase. : {'variables': {'input': ''}}


In [27]:
# Execute the plan.
result = await sequential_plan.invoke_async()

In [28]:
print(result)

SHE SELLS SEASHELLS BY THE SEASHORE


## Action Planner

Action planner takes in a list of functions and the goal, and outputs a single function that is appropriate to meet the goal.

In [3]:
from semantic_kernel.planning import ActionPlanner

planner = ActionPlanner(kernel)

DEBUG:root:Extracting blocks from template: A planner takes a list of functions, a goal, and chooses which function to use.
For each function the list includes details about the input parameters.
[START OF EXAMPLES]
{{ActionPlanner_Excluded.GoodExamples}}
{{ActionPlanner_Excluded.EdgeCaseExamples}}
[END OF EXAMPLES]
[REAL SCENARIO STARTS HERE]
- List of functions:
{{ActionPlanner_Excluded.ListOfFunctions}}
- End list of functions.
Goal: {{ $input }}

DEBUG:root:Importing skill ActionPlanner_Excluded
DEBUG:root:Methods imported: 3


In [4]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill

kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

DEBUG:root:Importing skill math
DEBUG:root:Methods imported: 2
DEBUG:root:Importing skill fileIO
DEBUG:root:Methods imported: 2
DEBUG:root:Importing skill time
DEBUG:root:Methods imported: 19
DEBUG:root:Importing skill text
DEBUG:root:Methods imported: 5


{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10f3ee010>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10f3ee410>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10f3ee790>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10f3eeb10>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10f3eee90>}

In [5]:
plan = await planner.create_plan_async(goal="what is the 1.23 + 3.21?")
plan

DEBUG:root:Rendering string template: A planner takes a list of functions, a goal, and chooses which function to use.
For each function the list includes details about the input parameters.
[START OF EXAMPLES]
{{ActionPlanner_Excluded.GoodExamples}}
{{ActionPlanner_Excluded.EdgeCaseExamples}}
[END OF EXAMPLES]
[REAL SCENARIO STARTS HERE]
- List of functions:
{{ActionPlanner_Excluded.ListOfFunctions}}
- End list of functions.
Goal: {{ $input }}

DEBUG:root:Extracting blocks from template: A planner takes a list of functions, a goal, and chooses which function to use.
For each function the list includes details about the input parameters.
[START OF EXAMPLES]
{{ActionPlanner_Excluded.GoodExamples}}
{{ActionPlanner_Excluded.EdgeCaseExamples}}
[END OF EXAMPLES]
[REAL SCENARIO STARTS HERE]
- List of functions:
{{ActionPlanner_Excluded.ListOfFunctions}}
- End list of functions.
Goal: {{ $input }}

DEBUG:root:Rendering list of 9 blocks
DEBUG:root:Rendering code: `ActionPlanner_Excluded.GoodExa

PlanningException: (<ErrorCodes.InvalidPlan: 1>, 'No valid plan has been generated.', ValueError("I don't know."))

In [ ]:
result = await plan.invoke_async()
print(result)

## Stepwise Planner

In [3]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

In [9]:
class FooSearch:
    """
    A search engine skill.
    """

    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import (
        sk_function,
        sk_function_context_parameter,
    )

    def __init__(self):
        pass

    @sk_function(
        description="Performs a Foo search for a given query", name="searchAsync"
    )
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        print("query is", query)
        return "wohoo"

In [4]:
# kernel.import_skill(FooSearch(), skill_name="FooSearch")
from semantic_kernel.connectors.search_engine import GoogleConnector
from semantic_kernel.core_skills import WebSearchEngineSkill

connector = GoogleConnector(
    api_key=os.getenv("GOOGLE_API_KEY"),
    search_engine_id=os.getenv("GOOGLE_SEARCH_ENGINE_ID"),
)

# Create a WebSearchEngineSkill and pass the Google Connector to it.
web_skill = kernel.import_skill(WebSearchEngineSkill(connector), "WebSearch")

DEBUG:root:Importing skill WebSearch
DEBUG:root:Methods imported: 1


In [5]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

ask = """How many total championships combined do the top 5 teams in the NBA have?"""

plan = planner.create_plan(goal=ask)

DEBUG:root:Extracting blocks from template: [INSTRUCTION]
Answer the following questions as accurately as possible using the provided functions.

[AVAILABLE FUNCTIONS]
The function definitions below are in the following format:
<functionName>: <description>
  inputs:
    - <parameterName>: <parameterDescription>
    - ...

{{$function_descriptions}}
[END AVAILABLE FUNCTIONS]

[USAGE INSTRUCTIONS]
To use the functions, specify a JSON blob representing an action. The JSON blob should contain an "action" key with the name of the function to use, and an "action_variables" key with a JSON object of string values to use when calling the function.
Do not call functions directly; they must be invoked through an action.
The "action_variables" value should always include an "input" key, even if the input value is empty. Additional keys in the "action_variables" value should match the defined [PARAMETERS] of the named "action" in [AVAILABLE FUNCTIONS].
Dictionary values in "action_variables" must

In [6]:
result = await plan.invoke_async()
print(result)

DEBUG:root:Rendering string template: [INSTRUCTION]
Answer the following questions as accurately as possible using the provided functions.

[AVAILABLE FUNCTIONS]
The function definitions below are in the following format:
<functionName>: <description>
  inputs:
    - <parameterName>: <parameterDescription>
    - ...

{{$function_descriptions}}
[END AVAILABLE FUNCTIONS]

[USAGE INSTRUCTIONS]
To use the functions, specify a JSON blob representing an action. The JSON blob should contain an "action" key with the name of the function to use, and an "action_variables" key with a JSON object of string values to use when calling the function.
Do not call functions directly; they must be invoked through an action.
The "action_variables" value should always include an "input" key, even if the input value is empty. Additional keys in the "action_variables" value should match the defined [PARAMETERS] of the named "action" in [AVAILABLE FUNCTIONS].
Dictionary values in "action_variables" must be st

KernelException: (<ErrorCodes.FunctionInvokeError: 8>, 'Error occurred while running plan step: (<ErrorCodes.UnknownError: -1>, "Error occurred while executing stepwise plan: (<ErrorCodes.ServiceError: 6>, \'Google PaLM service failed to complete the prompt\', InvalidArgument(\'Request payload size exceeds the limit: 20000 bytes.\'))", AIException(<ErrorCodes.ServiceError: 6>, \'Google PaLM service failed to complete the prompt\', InvalidArgument(\'Request payload size exceeds the limit: 20000 bytes.\')))', PlanningException(<ErrorCodes.UnknownError: -1>, "Error occurred while executing stepwise plan: (<ErrorCodes.ServiceError: 6>, 'Google PaLM service failed to complete the prompt', InvalidArgument('Request payload size exceeds the limit: 20000 bytes.'))", AIException(<ErrorCodes.ServiceError: 6>, 'Google PaLM service failed to complete the prompt', InvalidArgument('Request payload size exceeds the limit: 20000 bytes.'))))

In [7]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan


NameError: name 'result' is not defined